

## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from files on the local drive.  All data files can be downloaded from https://www.cse.ust.hk/msbd5003/data/
For example, https://www.cse.ust.hk/msbd5003/data/fruits.txt

In [1]:
import math
a = sc.parallelize(range(1,100000))
b = a.map(lambda x:math.sqrt(x))
b.count()

99999

In [2]:
logFile = "../../README.md"  # Should be some file on your system
logData = sc.textFile(logFile)

numAs = logData.filter(lambda s: 'a' in s).count()
numBs = logData.filter(lambda s: 'b' in s).count()

print("Lines with a:", numAs, " lines with b: ", numBs)

Lines with a: 64  lines with b:  32


In [3]:
# Read data from local file system:
print(sc.version)

fruits = sc.textFile('fruits.txt')
yellowThings = sc.textFile('yellowthings.txt')
print(fruits.collect())
print(yellowThings.collect())

3.0.3
['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']
['banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [5]:
# Read data from HDFS :
fruits = sc.textFile('fruits.txt')
fruits.collect()

['apple',
 'banana',
 'canary melon',
 'grap',
 'lemon',
 'orange',
 'pineapple',
 'strawberry']

----------

##  RDD operations

In [9]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [11]:
# fruitsReversed = fruits.map(lambda fruit: fruit[::-1])
fruitsReversed.persist()
# try changing the file and re-execute with and without cache
print(fruitsReversed.collect()) #如果persist（）被注释掉后，文件改变了，再次只运行改行代码输出的结果也会不同（因为再次执行DAG）
# What happens when you uncomment the first line and run the whole program again with cache()?

['elppa', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [12]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)
print(shortFruits.collect())

['apple', 'grap', 'lemon']


In [13]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))#One in, any out
print(characters.collect())

['a', 'p', 'p', 'l', 'e', 'b', 'a', 'n', 'a', 'n', 'a', 'c', 'a', 'n', 'a', 'r', 'y', ' ', 'm', 'e', 'l', 'o', 'n', 'g', 'r', 'a', 'p', 'l', 'e', 'm', 'o', 'n', 'o', 'r', 'a', 'n', 'g', 'e', 'p', 'i', 'n', 'e', 'a', 'p', 'p', 'l', 'e', 's', 't', 'r', 'a', 'w', 'b', 'e', 'r', 'r', 'y']


In [14]:
# union
fruitsAndYellowThings = fruits.union(yellowThings) #这个只是单纯并集，允许有重复元素出现 #concatnation
print(fruitsAndYellowThings.collect())

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [15]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
print(yellowFruits.collect())

['pineapple', 'canary melon', 'lemon', 'banana']


In [16]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
print(distinctFruitsAndYellowThings.collect())

['orange', 'pineapple', 'canary melon', 'lemon', 'bee', 'banana', 'butter', 'gold', 'sunflower', 'apple', 'grap', 'strawberry']


### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [17]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']


In [18]:
# count
numFruits = fruits.count()
print(numFruits)

8


In [19]:
# take
first3Fruits = fruits.take(3)
print(first3Fruits)

['apple', 'banana', 'canary melon']


In [20]:
print(fruits.map(lambda fruit: len(fruit)).sum())
#Must be Associative Operator (Dont care about the position)
print(fruits.map(lambda fruit: len(fruit)).reduce(lambda x,y : x+y)) #两两操作后结果和另外item操作 #和mapreduce不同，那个是reduct by key

57
57


In [21]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y)) #union is python union (distinct)
print(letterSet)

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [22]:
letterSet = fruits.flatMap(lambda fruit: list(fruit)).distinct().collect()
print(letterSet)

['p', 'l', 'b', 'c', 'r', 'y', 'g', 'i', 's', 'a', 'e', 'n', ' ', 'm', 'o', 't', 'w']


### Closure

In [1]:
counter = 0
rdd = sc.parallelize(range(10))

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x

print(rdd.collect())
rdd.foreach(increment_counter) #another action

print(counter)
print(rdd.sum())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
45


In [24]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x

a = rdd.foreach(g)

print(accum.value)

45


In [28]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0) #avoid

def g(x):
    global accum
    accum += x
    return x * x

a = rdd.map(g) # = foreach
print(accum.value)

a.reduce(lambda x, y: x+y) #action make accum+
print(accum.value)

a.cache()
tmp = a.count()
print(accum.value)

rdd.reduce(lambda x, y: x+y)

tmp = a.count()
print(accum.value)
# print(rdd.reduce(lambda x, y: x+y))


0
45
90
90


### Computing Pi using Monte Carlo simulation

In [16]:
# From the official spark examples.
import random
import time

partitions = 100
n = 10000 * partitions

def f(_):
    x = random.random()
    y = random.random()
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = sc.parallelize(range(1, n + 1), partitions).map(f).sum()

print("Pi is roughly", 4.0 * count / n) #求正方形中落入圆中点的比例

Pi is roughly 3.141436


In [20]:
# Example: glom
import sys
import random

def f(_):
    return random.random()

a = sc.parallelize(range(0,10),10)
print(a.collect())
print(a.glom().collect()) #glom() convert RDD to list of lists
print(a.map(f).glom().collect())

# Weird behavior: Initially, random numbers are synched across all workers, but will get 
# out-of-sync after a large (e.g, 1000000) number of random numbers have been generated.

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
[[0.6701936982081924], [0.09669661577798527], [0.6014414561231274], [0.332997328120584], [0.22897696253426958], [0.46577948640799915], [0.8467279083466749], [0.8181700240513209], [0.46102514877904677], [0.26939396481088207]]


In [73]:
# Example: mapPartition and mapPartitionWithIndex
a = sc.parallelize(range(0,20),4)
print(a.glom().collect())

def f(it):
    s = 0
    for i in it: #this for loop will only trigger when an action call it.
        s += i
        yield s #without construct list explicitly. Everything yielded will be append into a list

print(a.mapPartitions(f).collect()) #map based on each partition

def f(index, it): #index retrieves the number of partition (starts with 0)
    s = index
    for i in it:
        s += i
        yield s

print(a.mapPartitionsWithIndex(f).collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]
[0, 1, 3, 6, 10, 5, 11, 18, 26, 35, 10, 21, 33, 46, 60, 15, 31, 48, 66, 85]
[0, 1, 3, 6, 10, 6, 12, 19, 27, 36, 12, 23, 35, 48, 62, 18, 34, 51, 69, 88]


In [30]:
# Correct version
import random
import time

partitions = 100
n = 10000 * partitions

seed = time.time()

def f(index, it):
    random.seed(index + seed) #ensure every work use different seed
    for i in it:
        x = random.random()
        y = random.random()
        yield 1 if x ** 2 + y ** 2 < 1 else 0

count = sc.parallelize(range(1, n + 1), partitions) \
          .mapPartitionsWithIndex(f).sum() #different seeds fro diffrent partition

print("Pi is roughly", 4.0 * count / n)

Pi is roughly 3.144788


### Closure and Persistence

In [31]:
# RDD variables are references
A = sc.parallelize(range(10)) #2*x + 1
B = A.map(lambda x: x*2)
A = B.map(lambda x: x+1)
A.take(10)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [7]:
# Linear-time selection

data = [34, 67, 21, 56, 47, 89, 12, 44, 74, 43, 26]
A = sc.parallelize(data,8)
k = 4

while True:
    x = A.first()
    A1 = A.filter(lambda z: z < x)
    A2 = A.filter(lambda z: z > x)
    # print('X:',x)
#     print('K:',k)
#     print('A:',A.collect())
#     print('A1:',A1.collect())
    # print('A2:',A2.collect())
#     print('----------')
    mid = A1.count() 
    if mid == k:
        print(x)
        break
    if k < mid:
        A = A1
    else:
        A = A2
        k = k - mid - 1
    A.cache() #Actually it caches the A2 part, while the A1 part is useless

43


In [2]:
sorted(data)

[12, 21, 26, 34, 43, 44, 47, 56, 67, 74, 89]

In [34]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
B.cache() #if comment it, the B will re-compute 
print(B.count())
x = 3
print(B.count())

5
5


In [65]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
# B.cache()
B.unpersist()
# print(B.take(10))
print(B.collect())

x = 3
#print(B.take(10))
print(B.collect())
# collect() doesn't always re-collect data - bad design!
# Always use take() instead of collect()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


### Key-Value Pairs

In [35]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y) #相同的key的value相加，相当于word count
print(numFruitsByLength.take(10))

[(6, 2), (12, 1), (4, 1), (10, 1), (5, 2), (9, 1)]


In [36]:
# Word Count
from operator import add

lines = sc.textFile('course.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
# counts.collect()
print(counts.sortByKey().take(20))

[('Big', 1), ('Course', 2), ('Description', 1), ('Information', 1), ('Lecture', 1), ('This', 1), ('across', 1), ('amount', 1), ('and', 3), ('as', 1), ('both', 1), ('centers.', 1), ('cloud', 1), ('commodity', 1), ('computing', 1), ('course', 1), ('data', 4), ('emerge', 1), ('enabling', 1), ('even', 1)]


In [74]:
print(counts.sortBy(lambda x: x[1], False).take(20))

[('data', 4), ('of', 3), ('and', 3), ('Course', 2), ('in', 2), ('the', 2), ('Information', 1), ('systems,', 1), ('cloud', 1), ('parallel', 1), ('as', 1), ('mining', 1), ('massive', 1), ('amount', 1), ('even', 1), ('servers', 1), ('centers.', 1), ('both', 1), ('hands-on', 1), ('this', 1)]


In [37]:
# Join simple example

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
#trans = sc.parallelize([(1, 134, "OK"), (3, 34, "OK"), (5, 162, "Error"), (1, 135, "OK"), (2, 53, "OK"), (1, 45, "OK")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(products.join(trans).take(20)) #for key = 1,product three matchings in the result

[(1, ('Apple', (134, 'OK'))), (1, ('Apple', (135, 'OK'))), (1, ('Apple', (45, 'OK'))), (2, ('Orange', (53, 'OK'))), (3, ('TV', (34, 'OK'))), (5, ('Computer', (162, 'Error')))]


### K-means clustering

In [38]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split()])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_data.txt
lines = sc.textFile('kmeans_data.txt', 5)  

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_bigdata.txt
# lines = sc.textFile('../data/kmeans_bigdata.txt', 5)  
# lines is an RDD of strings
K = 3
convergeDist = 0.01  
# terminate algorithm when the total distance from old center to new centers is less than this value

data = lines.map(parseVector).cache() # data is an RDD of arrays

kCenters = data.takeSample(False, K, 1)  # intial centers as a list of arrays
tempDist = 1.0  # total distance from old centers to new centers

while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kCenters), (p, 1)))
    # for each point in data, find its closest center
    # closest is an RDD of tuples (index of closest center, (point, 1))
        
    pointStats = closest.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))
    # pointStats is an RDD of tuples (index of center,
    # (array of sums of coordinates, total number of points assigned))
    
    newCenters = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    # compute the new centers
    
    tempDist = sum(np.sum((kCenters[i] - p) ** 2) for (i, p) in newCenters)
    # compute the total disctance from old centers to new centers
    
    for (i, p) in newCenters:

        kCenters[i] = p
        
print("Final centers: ", kCenters)

Final centers:  [array([0.1       , 0.33333333, 0.23333333]), array([9.05, 3.05, 4.65]), array([9.2, 2.2, 9.2])]


### PageRank

In [39]:
import re
from operator import add

def computeContribs(urls, rank):
    # Calculates URL contributions to the rank of other URLs.
    #urls is a list of neighbour node
    #rank is the current weight of the current node
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)  #return a list of contribution

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/*
# lines = sc.textFile("pagerank_data.txt", 2)
lines = sc.textFile("dblp.in.txt", 5)

numOfIterations = 10

# Loads all URLs from input file and initialize their neighbors. 
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey() #ajacent list of the total graph

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0) #not change the key, the change the value part of the links
#(node,1.0)

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks) \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls, rank))
    
    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    # ranks = contribs.reduceByKey(add).map(lambda t: (t[0], t[1] * 0.85 + 0.15))

print(ranks.top(5, lambda x: x[1])) #the only action

[('3', 16775.735268839184), ('2', 15259.179382798724), ('1', 10159.700782343547), ('4', 8992.664552292099), ('5', 7293.343859690148)]


### Join vs. Broadcast Variables

In [79]:
products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(trans.join(products).take(20))


[(1, ((134, 'OK'), 'Apple')), (1, ((135, 'OK'), 'Apple')), (1, ((45, 'OK'), 'Apple')), (2, ((53, 'OK'), 'Orange')), (3, ((34, 'OK'), 'TV')), (5, ((162, 'Error'), 'Computer'))]


In [51]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

broadcasted_products = sc.broadcast(products) #better

results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
#  results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print(results.take(20))


[(1, 'Apple', (134, 'OK')), (3, 'TV', (34, 'OK')), (5, 'Computer', (162, 'Error')), (1, 'Apple', (135, 'OK')), (2, 'Orange', (53, 'OK')), (1, 'Apple', (45, 'OK'))]


In [118]:
from graphframes import *
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

# FILL IN YOUR CODE HERE

In [137]:
p = 4
rdd = sc.parallelize([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0], p)
                     
def find_longest_one(Index,it):
    
    start,index = 0,-1
    record_start,record_end,max_len = 0,0,0
    
    indicator = True #indicate previous element is 1 or not,default to 1
    
    current_len = 0 #length of consective 1s
    for i in it:
        index +=1 #record current index
        
        if i == 1:
            if indicator == True: #previous is 1
                current_len = index - start + 1
                
                if current_len > max_len:
                    record_start = start
                    record_end = index
                    max_len = current_len
                    
            else: #previous is not 1
                start = index
                indicator = True
                current_len = 0
            
        else:
            if indicator == True: #previous is 1
                indicator = False
                
            else: #previous is not 1
                current_len = 0
                
    # (Partition Index, nums of element, record start,record end, max len)     
    yield [Index,index+1,record_start,record_end,max_len] #

ans = rdd.mapPartitionsWithIndex(find_longest_one).collect()

previous_count = []
for i in ans:
    i[2] += sum(previous_count)
    i[3] += sum(previous_count)
    previous_count.append(i[1])

longest = 0
for i in range(1,len(ans)):
    tmp = ans[i][4]
    if ans[i-1][3] + 1 == ans[i][2]:
        tmp += ans[i-1][4]
        longest = max(longest,tmp)
    else:
        longest = max(longest,tmp)
longest

4

In [138]:
RDD1 = sc.parallelize([1,2,3,4])
RDD2 = sc.parallelize([1,2,3,4])

r1 = RDD1.zipWithIndex().map(lambda x: (x[1],x[0]))
r2 = RDD2.zipWithIndex().map(lambda x: (x[1],x[0]))
RDD3 = r1.union(r2).reduceByKey(lambda x,y: x*y).map(lambda x: x[1])
dot_product = RDD3.reduce(lambda x,y: x+y)

In [139]:
dot_product

30

In [140]:
r1.collect()

[(0, 1), (1, 2), (2, 3), (3, 4)]

In [141]:
RDD1.zipWithIndex().collect()

[(1, 0), (2, 1), (3, 2), (4, 3)]

In [142]:
RDD2.zipWithIndex().collect()

[(1, 0), (2, 1), (3, 2), (4, 3)]

In [145]:
r1.union(r2).reduceByKey(lambda x,y: x*y).collect()

[(0, 1), (1, 4), (2, 9), (3, 16)]